In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
import time
import copy

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Dataset directories
data_dir = './dataset/fruits_classification' 

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets
image_datasets = {x: datasets.ImageFolder(root=f'{data_dir}/{x}', transform=data_transforms[x]) for x in ['train', 'valid', 'test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'valid']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
class_names = image_datasets['train'].classes

# Function to visualize some images from the dataset
def visualize_dataset(dataset, num_images=15):
    # Get a batch of images and labels
    inputs, labels = next(iter(DataLoader(dataset, batch_size=num_images, shuffle=True)))
    
    # Move images to CPU for visualization if they're on GPU
    inputs = inputs.cpu()
    labels = labels.cpu()
    
    # Set up the plot
    plt.figure(figsize=(16, 12)) # Adjust size
    for i in range(num_images):
        ax = plt.subplot(3, 5, i + 1)  # 3 rows and 4 columns
        # Unnormalize the image
        image = inputs[i].numpy().transpose((1, 2, 0))  # Move channels to last dimension
        image = image * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])  # Unnormalize
        image = np.clip(image, 0, 1)  # Clip values to be between 0 and 1
        
        plt.imshow(image)
        plt.title(class_names[labels[i]])
        plt.axis('off')
    
    plt.show()

# Visualize some images from the training dataset
visualize_dataset(image_datasets['train'], num_images=15)

# Initialize model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Training function
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    train_loss_history, valid_loss_history = [], []
    train_acc_history, valid_acc_history = [], []

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss, running_corrects = 0.0, 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass and optimize only in train phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            # Step the scheduler if in train phase
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == 'train':
                train_loss_history.append(epoch_loss)
                train_acc_history.append(epoch_acc)
            else:
                valid_loss_history.append(epoch_loss)
                valid_acc_history.append(epoch_acc)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_loss_history, valid_loss_history, train_acc_history, valid_acc_history

# Train the model
model, train_loss_history, valid_loss_history, train_acc_history, valid_acc_history = train_model(
    model, criterion, optimizer, scheduler, num_epochs=25
)

# Visualize the loss and accuracy history
plt.figure(figsize=(12, 4))

# Convert loss and accuracy history from tensors to numpy arrays
train_loss_history_np = np.array(train_loss_history)
valid_loss_history_np = np.array(valid_loss_history)
# Move tensors to CPU and convert to numpy arrays
train_acc_history_np = np.array([acc.cpu().numpy() for acc in train_acc_history])
valid_acc_history_np = np.array([acc.cpu().numpy() for acc in valid_acc_history])

plt.subplot(1, 2, 1)
plt.plot(train_loss_history_np, label='Train Loss')
plt.plot(valid_loss_history_np, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_acc_history_np, label='Train Accuracy')
plt.plot(valid_acc_history_np, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

plt.show()

# Model summary
print("Model Summary:")
print(model)

from torchsummary import summary
input_size = (3, 224, 224)  # 3 channels (RGB), 224x224 images

print("Model Summary:")
summary(model, input_size=input_size, device=str(device))

from sklearn.metrics import classification_report
# Evaluate the model and generate classification report
def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []

    with torch.no_grad():  # Disable gradient calculation
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move to device
            outputs = model(inputs)  # Forward pass
            _, preds = torch.max(outputs, 1)  # Get predicted classes
            
            all_preds.extend(preds.cpu().numpy())  # Store predictions
            all_labels.extend(labels.cpu().numpy())  # Store true labels

    return np.array(all_labels), np.array(all_preds)

# Evaluate on the validation set
y_true, y_pred = evaluate_model(model, dataloaders['valid'])

# Generate the classification report
report = classification_report(y_true, y_pred, target_names=class_names)
print(report)

# Save the trained model
torch.save(model.state_dict(), 'fruit_classification_model.pth')
print("Model saved as fruit_classification_model.pth")

from PIL import Image
import os
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Define class names on fruit classification dataset
class_names = ['Apple', 'Banana', 'Grape', 'Mango', 'Strawberry']

# Load the trained model
model = models.resnet18(pretrained=True)  # Initialize model
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(class_names))  # Adjust the final layer
model.load_state_dict(torch.load('fruit_classification_model.pth'))  # Load weights
model = model.to(device)
model.eval()  # Set the model to evaluation mode

# Data transformations (same as used during training)
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Function to predict the class and confidence of an image
def predict_image(image_path):
    image = Image.open(image_path)
    image_tensor = data_transforms(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(image_tensor)
        probabilities = torch.nn.functional.softmax(output, dim=1)  # Confidence scores
        top_confidence, predicted_class = torch.max(probabilities, 1)
    
    return predicted_class.item(), top_confidence.item() * 100, image  # Convert to percentage

# Function to predict and display results for all images in a folder
def predict_images_in_folder(folder_path):
    image_paths = [os.path.join(folder_path, img) for img in os.listdir(folder_path) if img.endswith(('.png', '.jpg', '.jpeg'))]
    
    # Display each image with its prediction and confidence
    plt.figure(figsize=(12, len(image_paths) * 2))
    for idx, image_path in enumerate(image_paths):
        predicted_class, confidence, image = predict_image(image_path)
        
        # Display image
        plt.subplot(len(image_paths), 1, idx + 1)
        plt.imshow(image)
        plt.title(f'Predicted Class: {class_names[predicted_class]} | Confidence: {confidence:.2f}')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Example usage
folder_path = './dataset/fruits_classification/img-predict' 
predict_images_in_folder(folder_path)